In [64]:
import pandas as pd

pd.set_option('display.max_columns', None)

df_main = pd.read_csv("criminalidad_anual_limpio_201019.csv")

df_main.head()

,Unnamed: 0,1 CONTRA LAS PERSONAS,11-Homicidios dolosos/asesinatos,111-Homicidios dolosos/asesinatos consumados,12-Lesiones,13-Malos tratos ámbito familiar,14-Otros contra las personas,2 CONTRA LA LIBERTAD,21-Malos tratos habituales en el ámbito familiar,22-Otros contra la libertad,3 LIBERTAD SEXUAL,31-Agresión sexual,32-Agresión sexual con penetración,33-Corrupción de menores o incapacitados,34-Pornografía de menores,35-Otros contra la libertad sexual,4 RELACIONES FAMILIARES,5 PATRIMONIO,51-Hurtos,52-Robos con fuerza en las cosas,521-Robos con fuerza en las cosas en el interior de vehículos,522-Robos con fuerza en viviendas,523-Robos con fuerza en establecimientos,53-Robos con violencia o intimidación,531-Robos con violencia en vía pública,532-Robos con violencia en viviendas,533-Robos con violencia en establecimientos,54-Sustracción de vehículos,55-Estafas,551-Estafas informáticas,56-Daños,57-Contra la propiedad intelectual/industrial,58-Blanqueo de capitales,59-Otros contra el patrimonio,6 SEGURIDAD COLECTIVA,61-Tráfico de drogas,62-Contra la seguridad vial,63-Otros contra la seguridad colectiva,7 FALSEDADES,8 ADMÓN PÚBLICA,9 ADMÓN JUSTICIA,10 ORDEN PÚBLICO,11 LEGISLACIÓN ESPECIAL,12 OTRAS INFRACCIONES PENALES,TOTAL INFRACCIONES PENALES,Unnamed: 45,Year
0,TOTAL NACIONAL,186110.0,1167.0,331.0,103341.0,77424.0,4178.0,125685.0,14575.0,111110.0,15319.0,8885.0,3079.0,369.0,866.0,2120.0,7288.0,1707144.0,700453.0,298098.0,108637.0,98326.0,32363.0,65874.0,25916.0,3223.0,5609.0,35105.0,327723.0,192375.0,219424.0,1807.0,295.0,58365.0,68749.0,16624.0,49869.0,2256.0,24529.0,416.0,32082.0,21877.0,306.0,9970.0,2199475.0,NaN,2019.0
1,ANDALUCÍA,37130.0,238.0,82.0,20624.0,15455.0,813.0,25751.0,2752.0,22999.0,2510.0,1462.0,401.0,92.0,132.0,423.0,1503.0,239711.0,91298.0,47542.0,17691.0,15349.0,6935.0,7302.0,4714.0,605.0,1163.0,6511.0,42358.0,23237.0,33332.0,439.0,82.0,10847.0,15094.0,4366.0,10270.0,458.0,3988.0,93.0,6728.0,4172.0,191.0,2450.0,339321.0,NaN,2019.0
2,ARAGÓN,4760.0,15.0,6.0,2407.0,2246.0,92.0,2276.0,285.0,1991.0,330.0,212.0,47.0,11.0,20.0,40.0,161.0,31651.0,10546.0,5471.0,1703.0,1568.0,737.0,1033.0,730.0,40.0,193.0,242.0,7976.0,4536.0,5122.0,34.0,3.0,1224.0,1360.0,351.0,972.0,37.0,446.0,3.0,799.0,570.0,2.0,225.0,42583.0,NaN,2019.0
3,ASTURIAS (PRINCIPADO DE),3368.0,18.0,7.0,1947.0,1344.0,59.0,1872.0,107.0,1765.0,193.0,121.0,28.0,3.0,11.0,30.0,99.0,19066.0,6780.0,2440.0,421.0,908.0,618.0,362.0,196.0,41.0,97.0,224.0,5577.0,3657.0,2824.0,18.0,1.0,840.0,1205.0,206.0,948.0,51.0,220.0,6.0,567.0,360.0,2.0,205.0,27163.0,NaN,2019.0
4,BALEARS (ILLES),8110.0,48.0,6.0,3774.0,4123.0,165.0,4718.0,526.0,4192.0,743.0,492.0,126.0,14.0,16.0,95.0,188.0,58274.0,26584.0,7883.0,2893.0,2598.0,965.0,1314.0,843.0,85.0,164.0,1429.0,13091.0,7702.0,5625.0,133.0,8.0,2207.0,2880.0,656.0,2051.0,173.0,716.0,11.0,1095.0,652.0,0.0,275.0,77662.0,NaN,2019.0


In [65]:
import pandas as pd

# Suponiendo que tus datos están en un DataFrame llamado 'df'
new_columns = {
    "Unnamed: 0" : "Regions", 
    "1 CONTRA LAS PERSONAS": "1_against_people",
    "11-Homicidios dolosos/asesinatos": "1.1_intentional_homicides_murders",
    "111-Homicidios dolosos/asesinatos consumados": "1.1.1_intentional_homicides_murders_committed",
    "12-Lesiones": "1.2_injuries",
    "13-Malos tratos ámbito familiar": "1.3_domestic_violence",
    "14-Otros contra las personas": "1.4_other_crimes_against_people",
    "2 CONTRA LA LIBERTAD": "2_against_freedom",
    "21-Malos tratos habituales en el ámbito familiar": "2.1_habitual_domestic_violence",
    "22-Otros contra la libertad": "2.2_other_crimes_against_freedom",
    "3 LIBERTAD SEXUAL": "3_sexual_freedom",
    "31-Agresión sexual": "3.1_sexual_assault",
    "32-Agresión sexual con penetración": "3.1.1_sexual_assault_with_penetration",
    "33-Corrupción de menores o incapacitados": "3.2_corruption_of_minors_or_disabled",
    "34-Pornografía de menores": "3.3_child_pornography",
    "35-Otros contra la libertad sexual": "3.4_other_crimes_against_sexual_freedom",
    "4 RELACIONES FAMILIARES": "4_family_relationships",
    "5 PATRIMONIO": "5_property_crimes",
    "51-Hurtos": "5.1_thefts",
    "52-Robos con fuerza en las cosas": "5.2_robbery_with_force",
    "521-Robos con fuerza en el interior de vehículos": "5.2.1_robbery_with_force_inside_vehicles",
    "522-Robos con fuerza en viviendas": "5.2.2_robbery_with_force_in_homes",
    "523-Robos con fuerza en establecimientos": "5.2.3_robbery_with_force_in_establishments",
    "53-Robos con violencia o intimidación": "5.3_robbery_with_violence_or_intimidation",
    "531-Robos con violencia en vía pública": "5.3.1_robbery_with_violence_in_public_ways",
    "532-Robos con violencia en viviendas": "5.3.2_robbery_with_violence_in_homes",
    "533-Robos con violencia en establecimientos": "5.3.3_robbery_with_violence_in_establishments",
    "54-Sustracción de vehículos": "5.4_vehicle_theft",
    "55-Estafas": "5.5_fraud",
    "551-Estafas informáticas": "5.5.1_computer_fraud",
    "56-Daños": "5.6_damage",
    "57-Contra la propiedad intelectual/industrial": "5.7_against_intellectual_industrial_property",
    "58-Blanqueo de capitales": "5.8_money_laundering",
    "59-Otros contra el patrimonio": "5.9_other_property_crimes",
    "6 SEGURIDAD COLECTIVA": "6_collective_security",
    "61-Tráfico de drogas": "6.1_drug_trafficking",
    "62-Contra la seguridad vial": "6.2_against_road_safety",
    "63-Otros contra la seguridad colectiva": "6.3_other_collective_security_crimes",
    "7 FALSEDADES": "7_forgeries",
    "8 ADMÓN PÚBLICA": "8_public_administration",
    "9 ADMÓN JUSTICIA": "9_justice_administration",
    "10 ORDEN PÚBLICO": "10_public_order",
    "11 LEGISLACIÓN ESPECIAL": "11_special_legislation",
    "12 OTRAS INFRACCIONES PENALES": "12_other_criminal_offenses",
    "TOTAL INFRACCIONES PENALES": "total_criminal_offenses",
    "Year": "year"
}

# Aplicar el nuevo esquema a los nombres de las columnas
df_main = df_main.rename(columns=new_columns)
df_main.drop("Unnamed: 45", axis=1, inplace=True)
df_main.columns = df_main.columns.str.replace(' ', '_').str.replace('-', '_')


In [66]:
df_temp = df_main.loc[:, df_main.columns != 'Regions']
df_temp = df_temp.astype(int)

In [67]:
df_main = pd.concat([df_main["Regions"], df_temp], axis=1)

df_main["year"].nunique()

10

In [68]:
df_main

,Regions,1_against_people,1.1_intentional_homicides_murders,1.1.1_intentional_homicides_murders_committed,1.2_injuries,1.3_domestic_violence,1.4_other_crimes_against_people,2_against_freedom,2.1_habitual_domestic_violence,2.2_other_crimes_against_freedom,3_sexual_freedom,3.1_sexual_assault,3.1.1_sexual_assault_with_penetration,3.2_corruption_of_minors_or_disabled,3.3_child_pornography,3.4_other_crimes_against_sexual_freedom,4_family_relationships,5_property_crimes,5.1_thefts,5.2_robbery_with_force,521_Robos_con_fuerza_en_las_cosas_en_el_interior_de_vehículos,5.2.2_robbery_with_force_in_homes,5.2.3_robbery_with_force_in_establishments,5.3_robbery_with_violence_or_intimidation,5.3.1_robbery_with_violence_in_public_ways,5.3.2_robbery_with_violence_in_homes,5.3.3_robbery_with_violence_in_establishments,5.4_vehicle_theft,5.5_fraud,5.5.1_computer_fraud,5.6_damage,5.7_against_intellectual_industrial_property,5.8_money_laundering,5.9_other_property_crimes,6_collective_security,6.1_drug_trafficking,6.2_against_road_safety,6.3_other_collective_security_crimes,7_forgeries,8_public_administration,9_justice_administration,10_public_order,11_special_legislation,12_other_criminal_offenses,total_criminal_offenses,year
0,TOTAL NACIONAL,186110,1167,331,103341,77424,4178,125685,14575,111110,15319,8885,3079,369,866,2120,7288,1707144,700453,298098,108637,98326,32363,65874,25916,3223,5609,35105,327723,192375,219424,1807,295,58365,68749,16624,49869,2256,24529,416,32082,21877,306,9970,2199475,2019
1,ANDALUCÍA,37130,238,82,20624,15455,813,25751,2752,22999,2510,1462,401,92,132,423,1503,239711,91298,47542,17691,15349,6935,7302,4714,605,1163,6511,42358,23237,33332,439,82,10847,15094,4366,10270,458,3988,93,6728,4172,191,2450,339321,2019
2,ARAGÓN,4760,15,6,2407,2246,92,2276,285,1991,330,212,47,11,20,40,161,31651,10546,5471,1703,1568,737,1033,730,40,193,242,7976,4536,5122,34,3,1224,1360,351,972,37,446,3,799,570,2,225,42583,2019
3,ASTURIAS (PRINCIPADO DE),3368,18,7,1947,1344,59,1872,107,1765,193,121,28,3,11,30,99,19066,6780,2440,421,908,618,362,196,41,97,224,5577,3657,2824,18,1,840,1205,206,948,51,220,6,567,360,2,205,27163,2019
4,BALEARS (ILLES),8110,48,6,3774,4123,165,4718,526,4192,743,492,126,14,16,95,188,58274,26584,7883,2893,2598,965,1314,843,85,164,1429,13091,7702,5625,133,8,2207,2880,656,2051,173,716,11,1095,652,0,275,77662,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,RIOJA (LA),1695,6,2,847,327,515,170,37,133,49,20,14,3,7,5,31,6555,2668,2043,327,489,363,220,135,17,48,165,308,86,1066,7,2,76,519,64,436,19,54,0,137,212,0,26,9448,2010
216,CIUDAD AUTÓNOMA DE CEUTA,833,5,1,393,71,364,277,221,56,18,9,0,3,3,3,31,3053,983,723,497,93,49,189,141,8,13,436,96,60,553,9,0,64,1381,1116,263,2,53,2,101,160,13,52,5974,2010
217,CIUDAD AUTÓNOMA DE MELILLA,1011,4,0,501,252,254,126,23,103,38,16,10,1,3,8,30,2810,1043,774,427,156,79,194,171,1,8,250,68,23,438,14,0,29,304,174,121,9,42,1,70,170,0,50,4652,2010
218,EN EL EXTRANJERO,158,2,2,17,53,86,70,17,53,33,3,8,5,15,2,29,20188,2427,674,142,19,51,177,138,5,6,125,16688,4647,28,1,5,63,6,6,0,0,226,1,13,2,0,8,20734,2010
